# Smart-G demo notebook for Objects

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import LambSurface, RoughSurface, CusForward
from smartg.atmosphere import AtmAFGL, AeroOPAC
from smartg.tools.luts import merge
from smartg.water import IOP_1
import numpy as np
#import sys
import imp

import smartg.geometry
from smartg.geometry import Vector, Point, Normal, Ray, BBox
import smartg.transform
from smartg.transform import Transform, Aff
import smartg.visualizegeo
from smartg.visualizegeo import receiver_view, Mirror, Plane, Spheric, Transformation, \
    Entity, Analyse_create_entity, LambMirror, Matte

from matplotlib.ticker import MaxNLocator
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm as cm
from smartg.tools.smartg_view import smartg_view, input_view
import math
from matplotlib.colors import LogNorm

# Quick Start

## Creation of objects

In [ ]:
# We want to create a simple case with a receiver and four heliostats (MirA, B, C and D).
# The receiver will be created a x = 1km, the first heliostat at x = 950m, the
# second heliostat at x = 900m and so on. The mirror will be oriented such that the
# direct rays reflected by the mirrors will totally reach the receiver.

# The solar zenith angle
solarDir = 14.3
# Create the heliostats MirA, MirB, MirC and MirD
wMx = 0.004725
wMy = 0.00642
MirA = Entity(name = "reflector", \
              materialAV = Mirror(reflectivity = 0.88), \
              materialAR = Matte(reflectivity = 0.), \
              geo = Plane( p1 = Point(-wMx, -wMy, 0.),
                           p2 = Point(wMx, -wMy, 0.),
                           p3 = Point(-wMx, wMy, 0.),
                           p4 = Point(wMx, wMy, 0.) ), \
              transformation = Transformation( rotation = np.array([0., 20.281725, 0.]), \
                                               translation = np.array([0.95, 0., 0.00517]) ))

MirB = Entity(MirA); MirC = Entity(MirA); MirD = Entity(MirA);
MirB.transformation = Transformation( rotation = np.array([0., 29.460753, 0.]), \
                                      translation = np.array([0.9, 0., 0.00517]) )
MirC.transformation = Transformation( rotation = np.array([0., 35.129831, 0.]), \
                                      translation = np.array([0.85, 0., 0.00517]) )
MirD.transformation = Transformation( rotation = np.array([0., 38.715473, 0.]), \
                                      translation = np.array([0.8, 0., 0.00517]) )

# Create the receiver Recept1
wRx = 0.006
wRy = 0.007
Recept1 = Entity(name = "receptor", TC = 0.0005, \
                 materialAV = Matte(reflectivity = 0.), \
                 materialAR = Matte(reflectivity = 0.), \
                 geo = Plane( p1 = Point(-wRx, -wRy, 0.),
                              p2 = Point(wRx, -wRy, 0.),
                              p3 = Point(-wRx, wRy, 0.),
                              p4 = Point(wRx, wRy, 0.) ), \
                 transformation = Transformation( rotation = np.array([0., -101.5, 0.]), \
                                                  translation = np.array([1., 0., 0.1065]) ))

# Create the list containing all the objects
listobjs = [MirA, MirB, MirC, MirD, Recept1]

## Run the simulation

In [ ]:
w2 = 120.
Pmin = [-w2, -w2, -0.5]
Pmax = [w2, w2, 121]
interval0 = [Pmin, Pmax]
aer = AeroOPAC('desert', 0.25, 550.)
pro = AtmAFGL('afglms', comp=[aer], P0 = 877, H2O=1.2)
m = Smartg(debug_photon=False, double = True).run( surf = LambSurface(ALB=0.25), 
    THVDEG=solarDir, NF=1e6, wl=550., NBPHOTONS=1e7, NBLOOP = 1e7, atm=pro, 
    progress=False, myObjects=listobjs, BEER=1, #le={'th_deg':  np.array([0, 15, 30, 45, 60, 80]),'phi_deg': 0.,},
    interval = interval0, IsAtm = 1, PHVDEG = 0.) #, cusForward=custumF,)

## How to show the results ?

In [ ]:
# Show the description of the simulation
m.print_info(show_attrs=True)

In [ ]:
# 3D print of objects, the notebook give only a given 2D print of the 3D one
fig1 = Analyse_create_entity(listobjs, Theta = solarDir)

In [ ]:
# print the flux distribution at the receiver
receiver_view(m, [wRx, wRy], logI=False, nameFile = '1H_02_test')